---

### Goals

- Subset and homogenize sample annotation table
- Write sample annotations to .tsv
- Subset raw data
- Write data observations to .tsv table


### Changelog

20250106: forked from `notebooks/manuscripts/biology-rbps-in-activation/KR20241111.1_data_loading.ipynb`

---

In [1]:
%cd ../../../..

/home/k.rooijers/Projects/OOPS_2023


/home/k.rooijers/.cache/pypoetry/virtualenvs/oops-2023-mKfZaHlF-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


---

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

In [3]:
OUTDIR = Path("pipeline_activation/")

---

### Data loading

Using `soeps` (old way)

```python
from soeps.prot.data import load_dataset, get_sample_table

data = load_dataset(DATASET)

sample_table = get_sample_table(data['data'].columns)
sample_table = sample_table.sort_values(['batch', 'library_type', 'crosslinked', 'activated', 'torin', 'donor_id']).reset_index(drop=True)
assert sample_table.groupby(['batch', 'library_type', 'crosslinked', 'activated', 'torin']).size().max() <= 3

sample_subset = (
    sample_table
    .sort_values(['batch', 'library_type', 'crosslinked', 'activated', 'torin', 'donor_id'])
    .reset_index(drop=True)
)
grouping = sample_subset.reset_index(drop=True).groupby(['batch', 'library_type', 'crosslinked', 'activated', 'torin'])
assert grouping.size().max() <= 3, "No dataset has more than 3 replicates, so far"
```

In [4]:
from glob import glob

In [5]:
fns = sorted(glob(str(OUTDIR / "raw/OOPS*hgnc_aggregated.tsv.gz")))

In [6]:
fns

['pipeline_activation/raw/OOPS_2022.hgnc_aggregated.tsv.gz',
 'pipeline_activation/raw/OOPS_2022_FP.hgnc_aggregated.tsv.gz']

In [7]:
tbls = [pd.read_csv(fn, sep="\t", header=0, index_col=0) for fn in fns]

In [8]:
tbls[0]

,01_43-3443_nCL_NA,02_43-3443_CL_NA,03_43-3443_nCL_3HCD3CD28,04_43-3443_CL_3HCD3CD28,05_43-5852_nCL_NA,06_43-5852_CL_NA,07_43-5852_nCL_3HCD3CD28,08_43-5852_CL_3HCD3CD28,09_43-6589_nCL_NA,10_43-6589_CL_NA,11_43-6589_nCL_3HCD3CD28,12_43-6589_CL_3HCD3CD28
Gene,,,,,,,,,,,,
HGNC:10011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:10018,15962850.0,10078500.0,736010.0,2045700.0,6172250.0,15081340.0,2545700.0,2161400.0,1828270.0,421190.0,1209400.0,24395900.0
HGNC:10018;HGNC:10061,NaN,NaN,NaN,NaN,1551300.0,2707500.0,NaN,NaN,854630.0,NaN,2183800.0,NaN
HGNC:10019,7136040.0,5410600.0,NaN,3905100.0,NaN,7068180.0,NaN,NaN,NaN,2286300.0,856220.0,15008400.0
HGNC:10021,903410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1897900.0,6644300.0
...,...,...,...,...,...,...,...,...,...,...,...,...
HGNC:9988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:9992,7533300.0,6349770.0,949860.0,0.0,3606160.0,20878500.0,12747500.0,3423000.0,NaN,876020.0,2756230.0,16026000.0
HGNC:9996,1706700.0,NaN,NaN,5169100.0,NaN,2030460.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
assert len(set(col for tbl in tbls for col in tbl.columns)) == sum(tbl.shape[1] for tbl in tbls)

Concatenate tables:

In [10]:
observations = pd.concat(tbls, axis=1)

observations.shape

(7532, 24)

In [11]:
observations = observations.loc[~observations.index.isna()]

Extract sample labels / annotations from column names:

In [12]:
# Fix typo in column naming
observations.columns = [
    col.replace("3HCD3CD29", "3HCD3CD28")
    for col in observations.columns
]

In [13]:
from soeps.prot.data import get_sample_table

In [14]:
from functools import reduce

from operator import or_ as setunion

In [15]:
sample_table = get_sample_table(observations.columns)

In [16]:
# relabel donor IDs (include batch info, renumber 1, 2, 3)

donor_newlabels = reduce(
    setunion,
    (
    sample_table
    .groupby('batch')
    [['batch', 'donor_id']]
    .apply(
        lambda subset: {
            k: "B" + str(sorted(set(sample_table['batch'])).index(subset['batch'].values[0])) + "D" + str(i)
            for (i, k) in enumerate(sorted(set(subset['donor_id'])))
        },
    )
).values.tolist())

sample_table['old_donor_id'] = sample_table['donor_id'].values

sample_table['donor_id'] = sample_table['donor_id'].replace(donor_newlabels)

In [17]:
sample_subset = (
    sample_table
    .groupby("torin").get_group("no")
    .sort_values(['batch', 'library_type', 'crosslinked', 'activated', 'torin', 'donor_id'])
    .reset_index(drop=True)
)
grouping = sample_subset.reset_index(drop=True).groupby(['batch', 'library_type', 'crosslinked', 'activated', 'torin'])
assert grouping.size().max() <= 3, "No dataset has more than 3 replicates, so far"

In [18]:
sample_subset.groupby([
    "library_type", "crosslinked", "activated", "torin", "batch",
]).size().rename("number_of_samples").to_frame()

number_of_samples
library_type crosslinked activated torin batch                            
OOPS         no          no        no    batch_2022_DDA                  3
                         yes       no    batch_2022_DDA                  3
             yes         no        no    batch_2022_DDA                  3
                         yes       no    batch_2022_DDA                  3
fullproteome no          no        no    batch_2022_DDA                  3
                         yes       no    batch_2022_DDA                  3
             yes         no        no    batch_2022_DDA                  3
                         yes       no    batch_2022_DDA                  3

#### Intensity values

In [19]:
peptide_information = pd.read_csv("pipeline_activation/raw/peptideinfo.tsv.gz", sep="\t", index_col=0, low_memory=False, header=0)

In [20]:
peptide_information = peptide_information.loc[~peptide_information.index.isna()]

In [21]:
assert (peptide_information.index == observations.index).all()

In [22]:
# Get data, log-transform
Y = observations[[col for col in sample_subset['sample_name']]].fillna(0.).values
with np.errstate(divide='ignore', invalid='ignore'):
    Yl = np.log10(np.ma.masked_less_equal(Y, 0.))
    
# Remove genes having missing data for all samples
w_valid = ~Yl.mask.all(axis=1)

# Remove genes detected by fewer than `MIN_PEPTIDES` distinct peptide sequences
w_valid &= peptide_information["peptide_count"] >= 2

In [23]:
w_valid.sum()

6089

Remove genes that correspond to "ambiguous-gene peptides" (that otherwise have an unambiguous gene)

In [24]:
unambiguous_gene_ids = set(peptide_information.index[~peptide_information.index.str.contains(";")])

In [25]:
other_gene_ids = set(peptide_information.index.to_series().str.split(";").map(set).map(lambda s: s & unambiguous_gene_ids).map(len).loc[lambda x: x == 0].index)

from collections import defaultdict

m = defaultdict(lambda: set())

for k in other_gene_ids:
    for x in k.split(";"):
        m[x].add(k)

combined_gene_ids = set([next(iter(x)) for x in m.values() if len(x) == 1])

In [26]:
len(unambiguous_gene_ids), len(other_gene_ids), len(combined_gene_ids)

(6105, 69, 52)

In [27]:
w_valid &= peptide_information.index.isin(unambiguous_gene_ids | combined_gene_ids)

In [28]:
w_valid.sum()

5561

In [29]:
is_missing = np.isclose(Y, 0, rtol=0, atol=1e-3)

Just report on missingness rates, when grouped by condition or donor

In [30]:
sample_subset.index

RangeIndex(start=0, stop=24, step=1)

In [31]:
grouping_libtype = sample_subset.groupby(['library_type'])

In [32]:
n_donor_per_libtype = {
    libtype: np.array([
        (~is_missing)[:, idxs].any(axis=1)
        for (k, idxs) in grouping_libtype.get_group((libtype, )).groupby(["donor_id"]).groups.items()
    ]).sum(axis=0)
    for libtype in ("OOPS", "fullproteome")
}

In [33]:
n_cond_with_atleast2_repl_per_libtype = {
    libtype: np.array([
        (~is_missing)[:, idxs].sum(axis=1) >= 2
        for (k, idxs) in grouping_libtype.get_group((libtype, )).groupby(["crosslinked", "activated"]).groups.items()
    ]).sum(axis=0)
    for libtype in ("OOPS", "fullproteome")
}

---

Load HGNC metadata:

In [34]:
hgnc_metadata = pd.read_csv("pipeline_activation/raw/hgnc_metadata.tsv.gz", sep="\t", header=0, index_col=0)

In [35]:
assert len(hgnc_metadata) == len(observations)

In [36]:
assert (hgnc_metadata.index == observations.index).all()

---

### Export files

- don't normalize
- don't log-transform
- only the samples for the sample_subset
    - make sample labels prettier (R can't accept leading numbers ...)
- don't filter out genes (but do report on being filtered, in extra column)
- add hgnc_id and symbol columns

In [37]:
new_sample_labels = sample_subset.apply(
    lambda row: "{library_type}_CL{crosslinked}_act{activated}_d{donor_id}_s{sample_id}".format(**row),
    axis=1,
)

In [38]:
assert len(new_sample_labels) == len(set(new_sample_labels))

Write samplesheet:

In [39]:
assert type(sample_subset.index) is pd.core.indexes.range.RangeIndex

In [40]:
outfn = str(OUTDIR / "samplesheet.tsv")

sample_subset['sample_label'] = new_sample_labels

sample_subset.to_csv(
    outfn,
    sep="\t", header=True, index=False,
)

Write intensity values:

In [41]:
detection_rate_df = pd.DataFrame(
    {
        ("n_donor_%s" % libtype): n_donor_per_libtype[libtype]
        for libtype in n_donor_per_libtype
    } | {
        ("n_cond_with_atleast2_repl_%s" % libtype): n_cond_with_atleast2_repl_per_libtype[libtype]
        for libtype in n_cond_with_atleast2_repl_per_libtype
    } 
)

In [42]:
outfn = str(OUTDIR / "intensity-values.tsv")

pd.concat([
    hgnc_metadata.reset_index()[['hgnc_ids', 'symbols']],
    pd.Series(w_valid.values, name='w_valid'),
    detection_rate_df,
    pd.DataFrame(Y, columns=new_sample_labels).astype(int),
], axis=1).to_csv(
    outfn,
    sep="\t", header=True, index=False,
)

---

*The end*

---

---

---